In [1]:
import geoengine as ge  

In [2]:
out_dir = "../test_data/vector_datasets"
datasets = {
    "study_area_a2": {
        "name": "Respect Study Area A2 Bounds",
        "description": "Respect Study Area A2 Bounds",
        "spatial_reference": "EPSG:4326",
        "path": "/mnt/panq/dbs_research_data/geo_data/RESPECT/StudyArea/Studyareas_A2_MinimumBoundin.geojson",
        "layer_name": "Studyareas_A2_MinimumBoundin",
        "vector_type": "MultiPolygon",
        "columns": {
            "id": "int"
        },
        "bounds": ge.BoundingBox2D(-80.111492, -4.371095 , -78.939427, -3.671742),
        "dataset_id": "ce47e31e-2bac-4185-b8f8-8f6909210ec4"
    },
    "Streets": {
        "name": "Ecuador Streets",
        "description": "Streets in Ecuador",
        "spatial_reference": "EPSG:4326",
        "path": "/mnt/panq/dbs_research_data/geo_data/RESPECT/Ecuador/ecuador_strassen_4326.geojson",
        "layer_name": "strassen",
        "vector_type": "MultiLineString",
        "columns": {
            "id": "int"
        },
        "bounds": ge.BoundingBox2D(-81.000000, -4.998916, -75.165707, 1.365791),
        "dataset_id": "67f68dca-cf27-49be-8612-49dfde9e68be"

    },
    "Province": {        
        "name": "Ecuador provinces",
        "description": "Provinces of Ecuador",
        "spatial_reference": "EPSG:4326",
        "path": "/mnt/panq/dbs_research_data/geo_data/RESPECT/Ecuador/provinzen_4326.geojson",
        "layer_name": "provinzen",
        "vector_type": "MultiPolygon",
        "columns": {
            "id": "int"
        },
        "bounds": ge.BoundingBox2D(-81.009090, -5.011824, -75.185413, 1.478300),
        "dataset_id": "b70e7d10-f7f1-4324-9e79-39b47822c8b2"
    },

}

In [3]:
def create_vector_layer(x):
    
    dataset_name = x['name']
    dataset_description = x['description']
    dataset_spatial_reference = x['spatial_reference']
    dataset_vector_tye = x['vector_type']
    dataset_path = x['path']
    dataset_layer_name = x['layer_name']
    bounds = x['bounds']
    dataset_id = x['dataset_id']

    columns = {k: ge.VectorColumnInfo(v, ge.UnitlessMeasurement()) for (k,v) in x['columns'].items()}

    data_type = ge.VectorDataType.from_string(dataset_vector_tye)

    dataset_result_descriptor = ge.VectorResultDescriptor(
        spatial_reference=dataset_spatial_reference,
        data_type=data_type,
        columns=columns,
        spatial_bounds=bounds,
        time_bounds=None,
        spatial_resolution=None,
    )

    int_columns = [k for (k,v) in columns.items() if v.data_type == "int"]

    loading_info = ge.api.OgrLoadingInfo(
        fileName = dataset_path,
        layerName = dataset_layer_name,
        dataType = data_type.to_api_enum(),
        time= ge.api.OgrSourceDatasetTimeType(type = "none"),
        int = int_columns,
        onError= ge.api.OgrOnError.IGNORE,        
    )

    metadata = ge.api.OgrMetadata(
        loadingInfo = loading_info,
        resultDescriptor = dataset_result_descriptor.to_api_dict(),
        type = "OgrMetaData"
    )
    
    dataset_properties = ge.DatasetProperties(
        dataset_id = dataset_id,
        name = dataset_name,
        description = dataset_description,
        source_operator="OgrSource",
    )

    dataset = ge.api.DatasetDefinition(
        properties = dataset_properties.to_api_dict(),
        metaData = metadata,
    )

    return dataset

In [4]:
import os
import json

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

for x in datasets.values():
    dataset_def = create_vector_layer(x)

    file_name = x['name'].replace(" ", "_").lower()
    
    with open(os.path.join(out_dir, f"{file_name}.json"), "w") as out_file:
        json.dump(dataset_def, out_file, indent=2)
        out_file.write("\n")